We are going to create a program called "Nier Storyteller". In short, it uses a message system similar to Nier Automata's to generate a one sentence summary before creating a short story.

Example:

{A foolish doll} {died in a world} {of ended dreams.} turns into "In a forgotten land where sunlight barely touched the ground, a little doll wandered through the remains of shattered dreams. Its porcelain face, cracked and wea..."

First, we start by importing Agentops and Anthropic

In [3]:
!pip install agentops
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.3/287.3 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.3
    Uninstalling psutil-5.9.3:
      Successfully uninstalled psutil-5.9.3
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2024.66.154055 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.5 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0,

Setup our generic default statements

In [4]:
from anthropic import Anthropic, AsyncAnthropic
import agentops
import os
import random  # We don't need this for agentops, we use this to generate a message later
from dotenv import load_dotenv

And set our API keys.

In [6]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "ANTHROPIC KEY HERE"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "AGENTOPS KEY HERE"

Now let's set the client as Anthropic and an AgentOps session!

In [7]:
client = Anthropic(api_key=ANTHROPIC_API_KEY)

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["anthropic-example"])

Let's set a default story as a script!

In [10]:
defaultstory = "In a forgotten land where sunlight barely touched the ground, a little doll wandered through the remains of shattered dreams. Its porcelain face, cracked and weathered, reflected the emptiness that hung in the air like a lingering fog. The doll's painted eyes, now chipped and dull, stared into the distance, searching for something—anything—that still held life. It had once belonged to a child who dreamt of endless adventures, of castles in the clouds and whispered secrets under starry skies. But those dreams had long since crumbled to dust, leaving behind nothing but a hollow world where even hope dared not tread. The doll, a relic of a life that had faded, trudged through the darkness, its tiny feet stumbling over broken wishes and forgotten stories. Each step took more effort than the last, as if the world itself pulled at the doll's limbs, weary and bitter. It reached a place where the ground fell away into an abyss of despair, the edge crumbling under its weight. The doll paused, teetering on the brink. It reached out, as though to catch a fading dream, but there was nothing left to hold onto. With a faint crack, its brittle body gave way, and the doll tumbled silently into the void. And so, in a world where dreams had died, the foolish little doll met its end. There were no tears, no mourning. Only the soft, empty echo of its fall, fading into the darkness, as the land of ended dreams swallowed the last trace of what once was."

We are almost done! Let's generate a one sentence story summary by taking 3 random sentence fragments and connecting them!

In [11]:
# Define the lists
first = [
    "A unremarkable soldier",
    "A lone swordsman",
    "A lone lancer",
    "A lone pugilist",
    "A dual-wielder",
    "A weaponless soldier",
    "A beautiful android",
    "A small android",
    "A double-crossing android",
    "A weapon carrying android",
]

second = [
    "felt despair at this cold world",
    "held nothing back",
    "gave it all",
    "could not get up again",
    "grimaced in anger",
    "missed the chance of a lifetime",
    "couldn't find a weakpoint",
    "was overwhelmed",
    "was totally outmatched",
    "was distracted by a flower",
    "hesitated to land the killing blow",
    "was attacked from behind",
    "fell to the ground",
]

third = [
    "in a dark hole beneath a city",
    "underground",
    "at the enemy's lair",
    "inside an empty ship",
    "at a tower built by the gods",
    "on a tower smiled upon by angels",
    "inside a tall tower",
    "at a peace-loving village",
    "at a village of refugees",
    "in the free skies",
    "below dark skies",
    "in a blood-soaked battlefield",
]

# Generate a random sentence
generatedsentence = (
    f"{random.choice(first)} {random.choice(second)} {random.choice(third)}."
)

And now to construct a stream/message! We set an example for the assistant now!

In [12]:
stream = client.messages.create(
    max_tokens=2400,
    model="claude-3-5-sonnet-20240620",  # Comma added here
    messages=[
        {
            "role": "user",
            "content": "Create a story based on the three sentence fragments given to you, it has been combined into one below.",
        },
        {
            "role": "assistant",
            "content": "{A foolish doll} {died in a world} {of ended dreams.}",
        },
        {"role": "assistant", "content": defaultstory},
        {
            "role": "user",
            "content": "Create a story based on the three sentence fragments  given to you, it has been combined into one below.",
        },
        {"role": "assistant", "content": generatedsentence},
    ],
    stream=True,
)

response = ""
for event in stream:
    if event.type == "content_block_delta":
        response += event.delta.text
    elif event.type == "message_stop":
        print(generatedsentence)
        print(response)

A weaponless soldier grimaced in anger inside an empty ship.


The cavernous hull of the abandoned starship echoed with the frustrated growls of Sergeant Maria Vega as she paced its empty corridors. Her hands, calloused from years of combat, clenched and unclenched at her sides, a habit born of the unsettling absence of her trusty plasma rifle. Just hours ago, she had been part of an elite squad tasked with investigating a distress signal from this very vessel, but a series of unfortunate events had left her separated from her team and stripped of her weapons.

The ship, once a bustling hub of interstellar commerce, now drifted lifelessly through the void, its systems offline and its crew mysteriously vanished. As Maria moved through the ghostly interior, her anger at her predicament slowly gave way to a creeping sense of dread. Without her weapons, she felt naked and vulnerable, acutely aware of every shadow and unexplained noise.

Rounding a corner, she froze as a metallic clang echo

Finally, we put the session to sleep! Go to your AgentOps dashboard and you should see this session reflected!

In [ ]:
agentops.end_session("Success")